<a href="https://colab.research.google.com/github/SewoongLee/addition_TF/blob/main/addition_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
input = '1'+'2'

In [2]:
T = len(input)

In [3]:
base = 5
vocabs = [str(i) for i in range(base)]

stoi = {s: i for i, s in enumerate(vocabs)}
itos = {i: s for i, s in enumerate(vocabs)}

stoi

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

In [4]:
import numpy as np
y = np.cos(2 * np.pi * np.arange(base*2) / base)

W_emb = np.zeros((base, base))
for row in range(base):
    for col in range(base):
        W_emb[row, col] = y[base+row-col]

assert W_emb.shape[0] == len(vocabs)
emb_dim = W_emb.shape[1]
W_emb

array([[ 1.        ,  0.30901699, -0.80901699, -0.80901699,  0.30901699],
       [ 0.30901699,  1.        ,  0.30901699, -0.80901699, -0.80901699],
       [-0.80901699,  0.30901699,  1.        ,  0.30901699, -0.80901699],
       [-0.80901699, -0.80901699,  0.30901699,  1.        ,  0.30901699],
       [ 0.30901699, -0.80901699, -0.80901699,  0.30901699,  1.        ]])

In [5]:
x = np.array([W_emb[stoi[token]] for token in input])
x

array([[ 0.30901699,  1.        ,  0.30901699, -0.80901699, -0.80901699],
       [-0.80901699,  0.30901699,  1.        ,  0.30901699, -0.80901699]])

In [6]:
head_count = 1
d_head = emb_dim
d_model = d_head * head_count
d_model

5

In [7]:
W_Q = np.zeros((emb_dim, d_model))
W_K = np.zeros((emb_dim, d_model))
W_V = np.array([[-3.2021, -0.2326,  2.8309,  2.1360, -1.2955],
                [3.2581,  1.4925, -1.2066, -3.7250, -0.2787],
                [-0.0522, -2.8364, -1.3058,  1.5192,  2.9546],
                [0.7191,  2.9639,  1.7879, -2.6599, -2.6595],
                [-3.1858,  1.1225,  3.0914,  1.1233, -2.6416]]).T

Q = x @ W_Q
K = x @ W_K
V = x @ W_V

assert Q.shape == (T, d_model)
assert K.shape == (T, d_model)
assert V.shape == (T, d_model)

In [8]:
import torch

M = torch.triu(torch.ones(T, T) * float('-inf'), diagonal=1)
M = M.masked_fill(torch.isnan(M), 0).numpy()
M


array([[  0., -inf],
       [  0.,   0.]], dtype=float32)

In [9]:
QK = (Q @ K.T) / np.sqrt(d_model)
weights = torch.softmax(torch.from_numpy(QK+M), dim=-1)
weights

tensor([[1.0000, 0.0000],
        [0.5000, 0.5000]], dtype=torch.float64)

In [10]:
x

array([[ 0.30901699,  1.        ,  0.30901699, -0.80901699, -0.80901699],
       [-0.80901699,  0.30901699,  1.        ,  0.30901699, -0.80901699]])

In [11]:
Attention = weights @ V
Attention # Caveat: LayerNorm is not applied.

tensor([[-1.0273,  5.3655, -6.8754,  8.0421,  2.3217],
        [ 3.0152,  0.5293, -5.4682,  5.7469,  5.4108]], dtype=torch.float64)

In [12]:
ff_dim = d_model
# ff_dim = d_model * 4

W_1 = np.array([[1.8305, -1.4206,  0.8389, -1.2164,  1.0526],
                [-0.7171, -0.2259,  0.8739, -1.7993, -1.8252],
                [-1.3302,  1.4185, -0.0461,  0.4895, -1.3567],
                [-0.0274,  1.1750, -1.6125,  1.8073,  0.1500],
                [1.7236, -0.7677, -1.2270,  1.2695,  1.4843]]).T
b_1 = np.zeros((ff_dim,))
print('W_1\n', W_1)
print('b_1\n', b_1)
assert W_1.shape == (d_model, ff_dim)
assert b_1.shape == (ff_dim,)

W_2 = np.array([[0.1974, -0.8905,  1.6609, -1.3888,  0.1512],
                [1.6468, -2.0534,  0.5210,  1.0784, -1.9426],
                [0.2980,  0.6291, -1.5538,  1.3649, -1.2030],
                [-2.2521,  1.4725, -1.6111,  0.3627,  1.0607],
                [-0.6408, -0.0834,  0.6112, -1.7972,  1.2625]]).T
b_2 = np.zeros((d_model,))
print('W_2\n', W_2)
print('b_2\n', b_2)
assert W_2.shape == (ff_dim, d_model)
assert b_2.shape == (d_model,)

FFN_X = torch.relu(torch.from_numpy(Attention.numpy() @ W_1 + b_1)) @ W_2 + b_2
assert FFN_X.shape == Attention.shape == (T, d_model)
print('FFN(X)')
print(FFN_X.round())

W_1
 [[ 1.8305 -0.7171 -1.3302 -0.0274  1.7236]
 [-1.4206 -0.2259  1.4185  1.175  -0.7677]
 [ 0.8389  0.8739 -0.0461 -1.6125 -1.227 ]
 [-1.2164 -1.7993  0.4895  1.8073  1.2695]
 [ 1.0526 -1.8252 -1.3567  0.15    1.4843]]
b_1
 [0. 0. 0. 0. 0.]
W_2
 [[ 0.1974  1.6468  0.298  -2.2521 -0.6408]
 [-0.8905 -2.0534  0.6291  1.4725 -0.0834]
 [ 1.6609  0.521  -1.5538 -1.6111  0.6112]
 [-1.3888  1.0784  1.3649  0.3627 -1.7972]
 [ 0.1512 -1.9426 -1.203   1.0607  1.2625]]
b_2
 [0. 0. 0. 0. 0.]
FFN(X)
tensor([[-26.,   9.,   9.,  13., -31.],
        [-24., -30.,  -4.,  36.,  -3.]], dtype=torch.float64)


In [13]:
W_L = W_emb.T  # Last linear layer that converts to logits
W_L

array([[ 1.        ,  0.30901699, -0.80901699, -0.80901699,  0.30901699],
       [ 0.30901699,  1.        ,  0.30901699, -0.80901699, -0.80901699],
       [-0.80901699,  0.30901699,  1.        ,  0.30901699, -0.80901699],
       [-0.80901699, -0.80901699,  0.30901699,  1.        ,  0.30901699],
       [ 0.30901699, -0.80901699, -0.80901699,  0.30901699,  1.        ]])

In [14]:
next_token_logits = (FFN_X @ W_L)[-1]
next_token_logits

tensor([-60.3339, -65.3861,  19.9230,  77.6992,  28.0977], dtype=torch.float64)

In [15]:
next_token = itos[torch.argmax(torch.softmax(next_token_logits, dim=-1)).item()]
next_token

'3'

In [16]:
# Result
print(f'input: {input} => next token: {next_token}')

input: 12 => next token: 3


### Test All Cases

In [18]:
def forward(input):
    x = np.array([W_emb[stoi[token]] for token in input])
    Q = x @ W_Q
    K = x @ W_K
    V = x @ W_V
    QK = (Q @ K.T) / np.sqrt(d_model)
    weights = torch.softmax(torch.from_numpy(QK+M), dim=-1)
    Attention = weights @ V
    FFN_X = torch.relu(torch.from_numpy(Attention.numpy() @ W_1 + b_1)) @ W_2 + b_2
    next_token_logits = (FFN_X @ W_L)[-1]
    next_token = itos[torch.argmax(torch.softmax(next_token_logits, dim=-1)).item()]
    return next_token

In [23]:
for a in range(base):
    for b in range(base):
        input = str(a)+str(b)
        output = forward(input)
        print(f'{input} => {output} ({(a+b)%base == stoi[output]})')

00 => 0 (True)
01 => 1 (True)
02 => 2 (True)
03 => 3 (True)
04 => 4 (True)
10 => 1 (True)
11 => 2 (True)
12 => 3 (True)
13 => 4 (True)
14 => 0 (True)
20 => 2 (True)
21 => 3 (True)
22 => 4 (True)
23 => 0 (True)
24 => 1 (True)
30 => 3 (True)
31 => 4 (True)
32 => 0 (True)
33 => 1 (True)
34 => 2 (True)
40 => 4 (True)
41 => 0 (True)
42 => 1 (True)
43 => 2 (True)
44 => 3 (True)
